In [1]:
import vapory as vap
import planetspy.utils as putil
import numpy as np

import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
recinst = putil.record()
recinst.load("cloud-30.plan.gz")

In [3]:
def find_viewing_window(recinst, zoom=1/1.5):
    return 10*zoom/(max(abs(np.max(recinst.loc[0])), abs(np.min(recinst.loc[1]))))

def scale_scene(scene, factor):
    return np.array(scene)*factor

def scale_scenes(scenes, factor):
    return [scale_scene(i, factor) for i in scenes]

def make_spheres(scene, tex):
    return [vap.Sphere(list(i[0:2]), i[3], tex) for i in scene]

def make_trail(scenes, tex, rad, index):
    return [vap.Cylinder(list(scenes[i][index][0:2]), list(scenes[i + 1][index][0:2]),
                         rad, tex) for i in range(len(scenes) - 1)]

def make_trails(scenes, tex, rad):
    trl = [make_trail(scenes, tex, rad, i) for i in range(len(scenes[0]))]
    ret = []
    for i in trl:
        ret.extend(i)
    return ret

In [7]:
def split_rec_scale_sparse(recinst, interval, factor):
    scenes = scale_scenes(recinst.loc[::interval], factor)
    ret_chunks = []
    for i in range(len(scenes[0]), len(scenes[-1]) - 1, -1):
        ret_chunks.append([k for k in scenes if len(k) == i])
    return ret_chunks

def make_renderable(chunks, spheretex, trailtex, trailrad, i):
    lens = np.array([len(i) for i in chunks])
    lens = np.cumsum(lens)
    objs = []
    for k in range(len(lens)):
        if lens[k] < i:
            objs.extend(make_trails(chunks[k], trailtex, trailrad))
        else:
            objs.extend(make_trails(chunks[k][:i-lens[k]], trailtex, trailrad))
            objs.extend(make_spheres(chunks[k][i-lens[k]], spheretex))
    return objs

In [8]:
tex = vap.Texture(vap.Pigment('color', [1,0,1]))
light = vap.LightSource([0, 0, 10], 'color', [1,1,1])
cam = vap.Camera('location', [0,0,10], 'look_at', [0, 0, 0])

factor = find_viewing_window(recinst, 0.4)
chunks = split_rec_scale_sparse(recinst, 1, factor)
objs = make_renderable(chunks, tex, tex, 0.005, len(recinst.loc) - 1)

scene = vap.Scene(cam, objects=[light, vap.Background("color", [1, 1, 1])] + objs)
test = scene.render("test.png", width = 1920, height = 1080, antialiasing=0.001)

IndexError: index 3 is out of bounds for axis 0 with size 3